<a href="https://colab.research.google.com/github/karnamohit/kranka_ucm/blob/master/tf1_test_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all the useful libraries...

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Checking the version of TensorFlow, NumPy...

In [2]:
#print('TensorFlow version info:\t',tf.__version__)
!pip show tensorflow
print(' ')
print('------------------------------------------------------------')
print(' ')
!pip show numpy
#print('NumPy version info:\t \t',np.__version__)

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: grpcio, termcolor, google-pasta, numpy, tensorflow-estimator, keras-applications, six, astor, wheel, tensorboard, gast, keras-preprocessing, wrapt, absl-py, protobuf
Required-by: stable-baselines, magenta, fancyimpute
 
------------------------------------------------------------
 
Name: numpy
Version: 1.16.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: yellowbrick, xgboost, xarray, wordcloud, umap-learn, torchvision, torchtext, torch, thinc, Theano, tflearn, tfds-nightly, tensorflow, tensorflow-probabili

**Making, processing raw data**

Make up data (2x2, sequentially indexed matrices)...

In [0]:
series_data = np.zeros((100,2,2), np.float64)        # "time"-series data for a 2x2 evolving matrix with 100 time-steps, randomly initialized
for i in range(series_data.shape[0]):
    series_data[i,:,:] = (i**0.5)*3 + 5
#print(series_data)
tsteps = int(series_data.shape[0])         # total number of time-steps in the series
tsteps_train = int(tsteps/2)               # number of time-steps used for training
mat_row = int(series_data.shape[1])
mat_col = int(series_data.shape[2])

In [0]:
#print(series_data[tsteps_train-2,:,:])
#print(series_data[tsteps_train-1,:,:])
#print(series_data[tsteps_train,:,:])

Define a sequence (multiple consecutive 2x2 matrices) size, ```len_seq```, to be fed in as input (equivalent to the amount of memory in previous time-steps, in our case), and the amount of overlap, ```n_overlap``` between any two adjacent input sequences...

In [0]:
len_seq = 4
n_overlap = 3

if (n_overlap >= len_seq):
    raise Exception("n_overlap must be less than len_seq")

Build tensor of sequences, call it ```seq_series_data```...

In [0]:
n_seq = int((tsteps_train - (len_seq - n_overlap)) / (len_seq - n_overlap)) # number of sequences of size len_seq
#print(n_seq)
seq_series_data = np.zeros((n_seq,len_seq,mat_row,mat_col), np.float64)
seq_series_data_pred = np.zeros((1,mat_row,mat_col), np.float64)
#print(seq_series_data_pred)
i = 0
#j = 0
k = 0
while (i <= n_seq):           # the training sequences start at time-step 1 (series_data[0] element) in this case
    try:
        for j in range(len_seq - 1):
            seq_series_data[k,j,:,:] = series_data[i+j,:,:]
    except IndexError:
        break
    #print(seq_series_data[k,:,:,:])
    #print(j)
    seq_series_data_pred = np.append(seq_series_data_pred, series_data[i+j+1:i+j+2,:,:], axis = 0)
    i += len_seq - n_overlap
    #j += 1
    k += 1
    #print(k,i)
seq_series_data_pred = np.delete(seq_series_data_pred, seq_series_data_pred[0,:,:], axis=0) # true output
#print(seq_series_data_pred)
#print(type(seq_series_data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### IGNORE THE FOLLOWING ```tf.Data``` CODE-BLOCKS FOR NOW

---

**Using ```tf.Data``` to manipulate shape of data fed to the model** 

Make a ```tf.placeholder``` object to represent input data, ```seq_series_data```...

In [0]:
inp_ph = tf.placeholder(seq_series_data.dtype, seq_series_data.shape)

Use ```tf.data``` for storing input data represented by the ```tf.placeholder``` object ```inp_ph```...

In [0]:
inp_slices = tf.data.Dataset.from_tensor_slices(inp_ph)
#next_slice_item = inp_slices.make_initializable_iterator()

Run a ```tf.Session()``` (within the context manager) to verify sequential printing of the evolving 2x2 matrix sequences...

In [0]:
with tf.Session() as sess:
    while True:
        try:
            for i in inp_slices:
                print(sess.run(i, feed_dict={inp_ph: seq_series_data}))
                #print(sess.run(next_slice_item.make_one_shot_iterator().get_next()))
        except tf.errors.OutOfRangeError:
            break

Define batches of the ```input_slices``` (saves memory when building the computational graph)

In [0]:
input_batches = input_slices.batches(N,drop_remainder=True)

Set up placeholders for input, true output, predicted output...

In [0]:
x = tf.placeholder(tf.float)

### CODE CONTINUES BELOW HERE

---

In [0]:
# test to check data-type and shape of seq_series_data, seq_series_data_pred, and a slice seq_series_data

print('shape of X (including the true output): ',seq_series_data[0,:-1,:,:].shape) 
print('no. of batches of X: ', seq_series_data.shape[0]) # 1 - prints the number of time-steps, each of these associated with a sequence of 2x2 matrices
#print(seq_series_data[0].shape)
print('shape of Y_pred and Y_true: ', seq_series_data_pred[0,:,:].shape)
print('no. of predictions (equal to the no. of batches of X): ', seq_series_data_pred.shape[0]) 
            # 2 - prints the number of time-steps, each of these assocaited with one 2x2 matrix (model truth), must match 1 above
print('shape of X (excluding the true output): ', seq_series_data[0,:-1,:,:].shape) # prints the sequence-size used per prediction

shape of X (including the true output):  (3, 2, 2)
no. of batches of X:  49
shape of Y_pred and Y_true:  (2, 2)
no. of predictions (equal to the no. of batches of X):  49
shape of X (excluding the true output):  (3, 2, 2)


**Declaring parameters and variables to be used in the model**

Construct input placeholders for the model...

In [0]:
X = tf.placeholder(tf.float64, shape=(None, seq_series_data[0,:-1,:,:].shape[0], seq_series_data[0,:-1,:,:].shape[1]), name='input')
            # raw input (3x2x2)
#x1 = tf.math.exp(X)    # model input (modified raw input) (3x2x2)
Y_pred = tf.placeholder(tf.float64, shape=(None, seq_series_data_pred[0,:,:].shape),name='model_output')
            # model output: predicting a 2x2 matrix 1 time-step at a time
Y_true = tf.placeholder(tf.float64, shape=(None, seq_series_data_pred[0,:,:].shape),name='true_output')   # true output: same shape as the model output

**Building a model**

Define activation functions, operations, etc. ...

In [0]:
# using "activation=None" (linear operation) in tf.layers.dense for hidden layer h_1

def 


'''# using a convolution operation (with a custom filter), conv_3D, for hidden layer h_2

# define a custom filter, filter_1, to be used with the one-shot (stride-less) convolution operation, conv_3D_oneshot
#     ignoring padding for now
def filter_1(d, h, w):
    # the latest sequence gets the highest contribution
    filter = np.zeros((d, h, w), np.float64)
    for i in range(d):
        for j in range(h):
            for k in range(w):
                filter[i,j,k] = np.sqrt(np.exp(d))
    return tf.convert_to_tensor(filter, dtype=tf.int32)

# define a (stride-less) function for the convolution operation
def conv_3D_oneshot(z, name="custom_conv_3D_1"):
    shape = z.get_shape().as_list()
    depth = shape[0].value
    height = shape[1].value
    width = shape[2].value
    filter = filter_1(depth, height, width)
    return tf.math.reduce_sum(tf.math.multiply(z, filter), 0)'''

Define architectural (hyper-)parameters...

In [0]:
# define layers with #(nodes/layer), n_units_i, connected by weight matrices, w_i, and bias matrices, b_i

n_inp_n = seq_series_data[0,:-1,:,:].shape[0]
            # no. of 2x2 matrices fed as input ("units" per INPUT layer). N.B.: "_n" denotes it is a tensor input.
n_units_1 = 4   # no. of units in the first hidden layer
n_units_1_conv = n_units_1
            # no. of (convolution) units in the output layer; must equal the number of units in the immediately preceding hidden layer
n_outp_n  = seq_series_data_pred[:1,:,:].shape[0]
            # number of (2x2) tensors the model must output (the same size as the sequence-size of the model output, 1 in the current case)

Build the layers...

In [0]:
with tf.name_scope("ann_1"):    # test ANN model, "ann_1"
    h_1 = tf.layers.dense(X, n_units_1, activation=None, use_bias=False, name="hidden_layer_1")    # first hidden layer (linear activation)
    f_1 = tf.get_variable("inp_to_outp_1", [1, 3], trainable=True, dtype=tf.float64)    # "filter" with zero strides, for "one-shot convolution"
    hf_1 = tf.Tensordot(f_n, h_1, axes = 1, name="hidden_funnel_layer_1")   # funneling layer, to reduce input dimensions to output dimensions 
    h_2 = tf.layers.dense(hf_1, n_outp_n, activation=None,use_bias=False, name="output_layer")    # output layer
    Y_pred = h_2    # model output

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Define the loss function...

In [0]:
with tf.name_scope("loss_ann_1"):   # loss function for the model "ann_1"
    loss_n = tf.losses.mean_squared_error(Y_true, Y_pred)   # element-wise MSE
    loss = tf.math.reduce_max(loss_n, axis=None)    # the maximum-valued element of the MSE tensor, computed above as "loss_n"

State the optimization algorithm to be used...

In [0]:
learn_rate = 0.1    # update variable, v, as: v_new = v_old + delta_v = v_old - learn_rate*grad_v(loss)

with tf.name_scope("opt_ann_1"):    # optimizer for "ann_1"
    opt = tf.train.GradientDescentOptimizer(learn_rate)   # use the gradient descent optimization algorithm
    loss_min = opt.minimize(loss)   # operation to minimize "loss"